# 安裝 fugle-trade 套件

In [1]:
# pip install 需要的套件
!pip install fugle-trade -U

     |████████████████████████████████| 2.6 MB 1.4 MB/s eta 0:00:01
  Using cached keyring-23.5.0-py3-none-any.whl (33 kB)
  Attempting uninstall: keyring
    Found existing installation: keyring 23.1.0
    Uninstalling keyring-23.1.0:
      Successfully uninstalled keyring-23.1.0
  Attempting uninstall: fugle-trade-core
    Found existing installation: fugle-trade-core 0.1.5
    Uninstalling fugle-trade-core-0.1.5:
      Successfully uninstalled fugle-trade-core-0.1.5
  Attempting uninstall: fugle-trade
    Found existing installation: fugle-trade 0.1.6
    Uninstalling fugle-trade-0.1.6:
      Successfully uninstalled fugle-trade-0.1.6


In [2]:
# 安裝富果交易 API sdk
from configparser import ConfigParser
from fugle_trade.sdk import SDK

from fugle_trade.order import OrderObject
from fugle_trade.constant import (APCode, Trade, PriceFlag, BSFlag, Action)

# 模擬環境測試

您在首次使用交易 API 取得正式環境的使用權限前，您會需要在模擬環境至少送出一筆委託

In [ ]:
## 取得 new 模擬環境設定檔並登入

config = ConfigParser()

config.read('./config.simulation.ini')

sdk = SDK(config)

sdk.login()

Enter esun account password:
··········
Please set a password for your new keyring: ··········
Please confirm the password: ··········
Enter cert password:
··········


In [ ]:
## 模擬金鑰下單測試
order = OrderObject(
    buy_sell = Action.Buy,
    price = 27.0,
    stock_no = "2884",
    quantity = 2,
)

sdk.place_order(order)

print("Your order has been placed successfully.")

Your order has been placed successfully.


In [ ]:
## 查看委託回報
sdk.get_order_results()

[{'apcode': '1',
  'avgPrice': '0',
  'bsFlag': 'R',
  'buysell': 'B',
  'celable': '1',
  'celqty': 0,
  'celqty_share': 0,
  'chgdate': '20220328',
  'chgtime': '173420921',
  'errcode': '00000000',
  'errmsg': '',
  'market': 'T',
  'matqty': 0,
  'matqty_share': 0,
  'odprice': '27.0',
  'orddate': '20220328',
  'ordno': '',
  'ordstatus': '1',
  'ordtime': '173420921',
  'orgqty': 2,
  'orgqty_share': 2000,
  'preordno': 'P0E0005E',
  'priceflag': '0',
  'stockno': '2884',
  'syscode': 'FA',
  'trade': '0',
  'workdate': '20220329'}]

In [ ]:
## 更改密碼
# sdk.reset_password()

# 正式環境

在 colab 環境因安全性考量，會需要增加 keyring 來設置密碼，這邊的第一個密碼是證券登入密碼，第四個則是憑證密碼，中間兩個則是相同的 keyring 密碼

In [8]:
# 取得正式環境設定檔並登入

config = ConfigParser()

config.read('./config.ini')

sdk = SDK(config)

sdk.login()

Enter esun account password:
··········
Please set a password for your new keyring: ··········
Please confirm the password: ··········
Enter cert password:
··········


## 下委託

In [9]:
# 下委託單
order = OrderObject(
    ap_code = APCode.Common,# 盤別
    buy_sell = Action.Buy,
    price_flag= PriceFlag.LimitDown, # 下跌停價買進
    price = None,
    stock_no = str(2884),
    quantity = 10,
)

sdk.place_order(order)

{'ord_date': '20220331',
 'ord_no': 'P0C8N5IW',
 'ord_time': '174534847',
 'ord_type': '1',
 'ret_code': '000000',
 'ret_msg': '',
 'work_date': '20220401'}

In [10]:
# 委託回報
orderResult = sdk.get_order_results()

# 查看委託回報狀況
import pandas as pd
pd.DataFrame(orderResult)

,work_date,ord_date,ord_time,ord_status,ord_no,pre_ord_no,stock_no,buy_sell,ap_code,price_flag,...,mat_qty,cel_qty,celable,err_code,err_msg,avg_price,bs_flag,org_qty_share,mat_qty_share,cel_qty_share
0,20220401,20220331,174534847,1,,P0C8N5IW,2884,B,1,2,...,0,0,1,00000000,,0,R,10000,0,0
1,20220401,20220331,173202813,1,,P0A8EB4K,2884,B,1,2,...,0,2,2,00000000,,0,R,2000,0,2000
2,20220331,20220331,083017724,2,60105,P0A8D83O,2884,B,1,2,...,0,10,2,00000000,,0.0,R,10000,0,10000


In [11]:
# 刪掉最新一筆委託單
sdk.cancel_order(orderResult[0])

{'ord_date': '20220331',
 'ord_time': '174544309',
 'ret_code': '000000',
 'ret_msg': ''}

# 簡易策略 demo code

In [12]:
# pip install fugle-realtime API library from https://github.com/fugle-dev/fugle-realtime-python

!pip install fugle-realtime


     |████████████████████████████████| 63 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [13]:
# 安裝 富果行情 API sdk
from fugle_realtime import HttpClient

# 請輸入自己的 apiToken，才能查看其他個股哦 ！！
api_client = HttpClient(api_token='demo')

In [14]:
# 安裝相關套件
import requests
import numpy as np
import pandas as pd
import datetime
import time

In [15]:
# 取得正式環境設定檔並登入

config = ConfigParser()

config.read('./config.ini')

sdk = SDK(config)

sdk.login()

## 觸量下單 demo



策略描述：<br>
盤中突然的大單，可以反映大戶的交易意願提升，而外盤成交代表買方願意用比較高的價格買，代表有較強烈的買進意圖<br>
因此這裡建構一個簡單的策略：當最新一筆成交是外盤成交，且成交量是前一筆的 10 倍以上就跟！

In [16]:
## set parameter

# 輸入股票代碼
symbolId = "2884"

# 下單張數
tradingVolume = 10


#----------------------------------
while True:  

    # 從 dealts API 取得最新兩筆成交明細數據 
    price_data = api_client.intraday.dealts(symbolId = symbolId)['data']['dealts'][:5]
 
    
    #策略條件：最新一筆成交量 大於等於 前一筆成交量的10倍 且為外盤成交
    if (price_data[0]['volume'] >= price_data[1]['volume']*10) and (price_data[0]['ask'] <= price_data[0]['price']):

        # 委託資訊
        order = OrderObject(
            buy_sell = Action.Buy,
            price_flag = PriceFlag.LimitDown,
            price = None,
            stock_no = str(symbolId),
            quantity = tradingVolume,
            ap_code = APCode.Common
        )

        # 送下單委託
        sdk.place_order(order)
        
        print('成功送出委託')
        print(f'買進張數：{tradingVolume} 張')

        break
        
    else:
        print(f"最新成交量 {str(price_data[0]['volume'])} 張")
        print(f'外盤成交：',str(price_data[0]['ask'] <= price_data[0]['price']))
        print('未達買進條件!')
        print('---')
        time.sleep(1)


成功送出委託
買進張數：10 張


In [17]:
# 查看委託回報
sdk.login()
orderResult = sdk.get_order_results()
print(orderResult[0])

{'work_date': '20220401', 'ord_date': '20220331', 'ord_time': '174638699', 'ord_status': '1', 'ord_no': '', 'pre_ord_no': 'P0B8KKS3', 'stock_no': '2884', 'buy_sell': 'B', 'ap_code': '1', 'price_flag': '2', 'trade': '0', 'od_price': '0.0', 'org_qty': 10, 'mat_qty': 0, 'cel_qty': 0, 'celable': '1', 'err_code': '00000000', 'err_msg': '', 'avg_price': '0', 'bs_flag': 'R', 'org_qty_share': 10000, 'mat_qty_share': 0, 'cel_qty_share': 0}


In [18]:
# 刪掉第 0 筆單
sdk.cancel_order(orderResult[0])

{'ord_date': '20220331',
 'ord_time': '174649364',
 'ret_code': '000000',
 'ret_msg': ''}